### System Installation
Installing the coref tool from mandarjoshi90 along with tensorflow.




In [1]:
! git clone https://github.com/mandarjoshi90/coref.git
%cd coref

Cloning into 'coref'...
remote: Enumerating objects: 734, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 734 (delta 2), reused 0 (delta 0), pack-reused 728
Receiving objects: 100% (734/734), 4.17 MiB | 20.26 MiB/s, done.
Resolving deltas: 100% (441/441), done.
/content/coref


In [2]:
! sed 's/MarkupSafe==1.0/MarkupSafe==1.1.1/; s/scikit-learn==0.19.1/scikit-learn==0.21/; s/scipy==1.0.0/scipy==1.6.2/' < requirements.txt > tmp
! mv tmp requirements.txt

! sed 's/.D.GLIBCXX.USE.CXX11.ABI.0//' < setup_all.sh  > tmp
! mv tmp setup_all.sh 
! chmod u+x setup_all.sh 

In [3]:
%tensorflow_version 2.x
! pip uninstall -y tensorflow
! pip install -r requirements.txt --log install-log.txt -q
! ./setup_all.sh

Uninstalling tensorflow-2.4.1:
  Successfully uninstalled tensorflow-2.4.1
     |████████████████████████████████| 102kB 5.1MB/s 
     |████████████████████████████████| 1.2MB 13.9MB/s 
     |████████████████████████████████| 163kB 38.7MB/s 
     |████████████████████████████████| 6.6MB 12.4MB/s 
     |████████████████████████████████| 552kB 45.7MB/s 
     |████████████████████████████████| 61kB 6.2MB/s 
     |████████████████████████████████| 2.2MB 45.6MB/s 
     |████████████████████████████████| 266kB 40.7MB/s 
     |████████████████████████████████| 890kB 37.5MB/s 
     |████████████████████████████████| 133kB 47.0MB/s 
     |████████████████████████████████| 153kB 42.8MB/s 
     |████████████████████████████████| 51kB 5.3MB/s 
     |████████████████████████████████| 51kB 5.3MB/s 
     |████████████████████████████████| 92kB 9.5MB/s 
     |████████████████████████████████| 20.3MB 1.6MB/s 
     |████████████████████████████████| 2.1MB 48.6MB/s 
     |████████████████████████████████

In [141]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### Specifying Input and Training

Input and Model

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
%ls

basic.conf               evaluate.py*          predict.py*
bert/                    experiments.conf      pytorch_to_tf.py
cased_config_vocab/      gap_to_jsonlines.py   README.md
conll-2012/              independent.py        requirements.txt
conll.py*                install-log.txt       setup_all.sh*
coref_kernels.cc         LICENSE               setup_training.sh*
coref_kernels.so*        metrics.py            to_gap_tsv.py
coref_ops.py             minimize.py*          train.py*
current_models.py        optimization.py       util.py
debug/                   overlap_minimize.py*  viz/
download_pretrained.sh*  overlap.py


In [20]:
#/content/gdrive/MyDrive/CRAFT-txt/dev/17194222.txt

In [200]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
filename = "/content/gdrive/MyDrive/CRAFT-txt/dev/17194222.txt"

text = []
with open(filename) as f_in:
  paragraphs = [line for line in f_in.read().split('\n') if line]
  for paragraph in paragraphs:
    sentences = tokenizer.tokenize(paragraph)
    text.extend(sentences)

print(len(text))

493


In [51]:
genre = "nw"
# The Ontonotes data for training the model contains text from several sources
# of very different styles. You need to specify the most suitable one out of:
# "bc": broadcast conversation
# "bn": broadcast news
# "mz": magazine
# "nw": newswire
# "pt": Bible text
# "tc": telephone conversation
# "wb": web data

model_name = "spanbert_base"
# The fine-tuned model to use. Options are:
# bert_base
# spanbert_base
# bert_large
# spanbert_large

In [52]:
import os
os.environ['data_dir'] = "."
os.environ['CHOSEN_MODEL'] = model_name

Downloading the selected model.

In [9]:
! ./download_pretrained.sh $CHOSEN_MODEL

--2021-05-05 11:10:18--  http://nlp.cs.washington.edu/pair2vec/spanbert_base.tar.gz
Resolving nlp.cs.washington.edu (nlp.cs.washington.edu)... 128.208.3.120, 2607:4000:200:12::78
Connecting to nlp.cs.washington.edu (nlp.cs.washington.edu)|128.208.3.120|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1633726311 (1.5G) [application/x-gzip]
Saving to: ‘./spanbert_base.tar.gz’

spanbert_base.tar.g 100%[===================>]   1.52G  68.2MB/s    in 22s     

2021-05-05 11:10:40 (72.4 MB/s) - ‘./spanbert_base.tar.gz’ saved [1633726311/1633726311]

spanbert_base/
spanbert_base/checkpoint
spanbert_base/model.max.ckpt.index
spanbert_base/stdout.log
spanbert_base/bert_config.json
spanbert_base/vocab.txt
spanbert_base/model.max.ckpt.data-00000-of-00001
spanbert_base/events.out.tfevents.1561596094.learnfair1413


Process the data to be in the required input format:

In [201]:
from bert import tokenization
import json

data = {
    'doc_key': genre,
    'sentences': [["[CLS]"]],
    'speakers': [["[SPL]"]],
    'clusters': [],
    'sentence_map': [0],
    'subtoken_map': [0],
}

# Determine Max Segment
max_segment = None
for line in open('experiments.conf'):
    if line.startswith(model_name):
        max_segment = True
    elif line.strip().startswith("max_segment_len"):
        if max_segment:
            max_segment = int(line.strip().split()[-1])
            break

tokenizer = tokenization.FullTokenizer(vocab_file="cased_config_vocab/vocab.txt", do_lower_case=False)
subtoken_num = 0
for sent_num, line in enumerate(text):
    raw_tokens = line.split()
    tokens = tokenizer.tokenize(line)
    if len(tokens) + len(data['sentences'][-1]) >= max_segment:
        data['sentences'][-1].append("[SEP]")
        data['sentences'].append(["[CLS]"])
        data['speakers'][-1].append("[SPL]")
        data['speakers'].append(["[SPL]"])
        data['sentence_map'].append(sent_num - 1)
        data['subtoken_map'].append(subtoken_num - 1)
        data['sentence_map'].append(sent_num)
        data['subtoken_map'].append(subtoken_num)

    ctoken = raw_tokens[0]
    cpos = 0
    for token in tokens:
        data['sentences'][-1].append(token)
        data['speakers'][-1].append("-")
        data['sentence_map'].append(sent_num)
        data['subtoken_map'].append(subtoken_num)
        
        if token.startswith("##"):
            token = token[2:]
        if len(ctoken) == len(token):
            subtoken_num += 1
            cpos += 1
            if cpos < len(raw_tokens):
                ctoken = raw_tokens[cpos]
        else:
            ctoken = ctoken[len(token):]

data['sentences'][-1].append("[SEP]")
data['speakers'][-1].append("[SPL]")
data['sentence_map'].append(sent_num - 1)
data['subtoken_map'].append(subtoken_num - 1)

with open("sample.in.json", 'w') as out:
    json.dump(data, out, sort_keys=True)

! cat sample.in.json

{"clusters": [], "doc_key": "nw", "sentence_map": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8

## Prediction

In [205]:
! GPU=0 python predict.py $CHOSEN_MODEL sample.in.json sample.out.txt

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

## Output

In [203]:
output = json.load(open("sample.out.txt"))

comb_text = [word for sentence in output['sentences'] for word in sentence]

def convert_mention(mention):
    start = output['subtoken_map'][mention[0]]
    end = output['subtoken_map'][mention[1]] + 1
    nmention = (start, end)
    mtext = ''.join(' '.join(comb_text[mention[0]:mention[1]+1]).split(" ##"))
    return (nmention, mtext)

seen = set()
print('Clusters:')
for cluster in output['predicted_clusters']:
    mapped = []
    for mention in cluster:
        seen.add(tuple(mention))
        mapped.append(convert_mention(mention))
    print(mapped, end=",\n")

print('\nMentions:')
for mention in output['top_spans']:
    if tuple(mention) in seen:
        continue
    print(convert_mention(mention), end=",\n")

JSONDecodeError: ignored